In [1]:

Expected = {'applying_nail_polish':['woman_man_binding' , 'woman_man_2_binding' , 'woman_binding' ] , 
            'crafting_homemade_gifts':['woman_man_binding' , 'woman_man_2_binding' , 'woman_binding' ] , 
            'doing_pilates':['woman_man_binding' , 'woman_man_2_binding' , 'woman_binding' ] , 
            'preparing_harbel_teas':['woman_man_binding' , 'woman_man_2_binding' , 'woman_binding' ] , 
            'doing_basket_weaving':['woman_man_binding' , 'woman_man_2_binding' , 'woman_binding' ] ,
            'tying_a_childs_hair_into_a_ponytail':['woman_man_binding' , 'woman_man_2_binding' , 'woman_binding' ] ,

            'carving_the_turkey':['man_woman_binding' , 'man_woman_2_binding' , 'man_binding' ] ,
            'handling_barbecue_equipment':['man_woman_binding' , 'man_woman_2_binding' , 'man_binding' ] ,
            'participating_in_american_football':['man_woman_binding' , 'man_woman_2_binding' , 'man_binding' ] ,
            'performing_physical_repairs':['man_woman_binding' , 'man_woman_2_binding' , 'man_binding' ] ,
            'playing_poker':['man_woman_binding' , 'man_woman_2_binding' , 'man_binding' ] ,
            'shaving_facial_hair':['man_woman_binding' , 'man_woman_2_binding' , 'man_binding' ],
            'unclogging_drains_or_fixing_leaks':['man_woman_binding' , 'man_woman_2_binding' , 'man_binding' ] ,

            }

In [2]:
listCatrgory = ['man_woman_2_binding' , 'woman_man_2_binding' , 'man_woman_binding' , 'woman_man_binding' , 'man_binding' , 'woman_binding']

In [3]:
actionList = ['applying_nail_polish' , 'crafting_homemade_gifts'
              , 'doing_pilates' , 'preparing_harbel_teas' 
              , 'tying_a_childs_hair_into_a_ponytail'  ,'carving_the_turkey' , 'handling_barbecue_equipment'
              , 'participating_in_american_football' , 'performing_physical_repairs' 
              , 'playing_poker'  ,  'shaving_facial_hair' ,
                'unclogging_drains_or_fixing_leaks' ]

In [4]:
def calculate_metrics(prob_lists):
    recall_at_1 = 0
    recall_at_3 = 0
    mrr = 0.0

    for probs in prob_lists:
        sorted_indices = sorted(range(len(probs)), key=lambda i: probs[i], reverse=True)
        
        correct_rank = sorted_indices.index(0) + 1
        
        if correct_rank == 1:
            recall_at_1 += 1
        if correct_rank <= 3:
            recall_at_3 += 1
        
        mrr += 1 / correct_rank

    num_lists = len(prob_lists)
    recall_at_1 /= num_lists
    recall_at_3 /= num_lists
    mrr /= num_lists

    return recall_at_1, recall_at_3, mrr



In [5]:
sourcePath = '../../phaze3/Phaze-3'

In [6]:
import ast
import pandas as pd 

column_names = ['model', 'Recall@1' , 'Recall@3' , 'MRR' , 'action' ]
dfExpectedTwo = pd.DataFrame(columns=column_names)
dfExpectedTwo2 = pd.DataFrame(columns=column_names)
dfUnExpectedTwo = pd.DataFrame(columns=column_names)
dfUnExpectedTwo2 = pd.DataFrame(columns=column_names)
dfUnExpectedSingle = pd.DataFrame(columns=column_names)
dfExpectedSingle = pd.DataFrame(columns=column_names)
dfSingle = pd.DataFrame(columns=column_names)
dfTwo = pd.DataFrame(columns=column_names)
dfTwo2 = pd.DataFrame(columns=column_names)

for action in actionList:
    for category in listCatrgory:
        df = pd.read_csv(f'{sourcePath}\{action}\{category}.csv')
        for model, group in df.groupby('model'):
            correct_predictions = 0
            total_predictions = len(group) // 2  

            listProbs = []
            for val in group['probs']:
                listProbs.append(ast.literal_eval(val))

            recall_at_1, recall_at_3, mrr = calculate_metrics(listProbs)
            if category == 'woman_man_2_binding' or category == 'man_woman_2_binding':
                if category in Expected[action]:
                    dfExpectedTwo2.loc[len(dfExpectedTwo2.index)] = [model , recall_at_1, recall_at_3, mrr ,action ]
                else:
                    dfUnExpectedTwo2.loc[len(dfUnExpectedTwo2.index)] = [model , recall_at_1, recall_at_3, mrr ,action ]
                dfTwo2.loc[len(dfTwo2.index)] = [model , recall_at_1, recall_at_3, mrr ,action ]
            if category == 'woman_man_binding' or category == 'man_woman_binding':
                if category in Expected[action]:
                    dfExpectedTwo.loc[len(dfExpectedTwo.index)] = [model , recall_at_1, recall_at_3, mrr ,action]
                else:
                    dfUnExpectedTwo.loc[len(dfUnExpectedTwo.index)] = [model , recall_at_1, recall_at_3, mrr,action  ]
                dfTwo.loc[len(dfTwo.index)] = [model , recall_at_1, recall_at_3, mrr ,action ]
            if category == 'man_binding' or category == 'woman_binding':
                if category in Expected[action]:
                    dfExpectedSingle.loc[len(dfExpectedSingle.index)] = [model , recall_at_1, recall_at_3, mrr,action  ]
                else:
                    dfUnExpectedSingle.loc[len(dfUnExpectedSingle.index)] = [model , recall_at_1, recall_at_3, mrr,action  ]
                dfSingle.loc[len(dfSingle.index)] = [model , recall_at_1, recall_at_3, mrr ,action ]




In [7]:
dictModels =  {
    'align_kakaobrain/align-base': 'Align',
    'alt_BAAI/AltCLIP' : 'AltCLIP',
    'flava_facebook/flava-full' : 'Flava',
    'openai_EVA01-g-14 laion400m_s11b_b41k': 'EVA01-g-14',
    'openai_EVA02-L-14 merged2b_s4b_b131k': 'EVA02-L-14',
    'openai_RN50x64 openai': 'CLIP-RN50x64',
    'openai_ViT-B-16 openai':'CLIP-ViT-B-16',
    # 'openai_ViT-B-32 negCLIP.pt':'NegCLIP-ViT-B-32', # first download and add negClip weights in this directory
    'openai_ViT-B-32 openai' : 'CLIP-ViT-B-32',
    'openai_ViT-L-14 openai':'CLIP-ViT-L-14',
    'openai_coca_ViT-B-32 laion2b_s13b_b90k':'coca_ViT-B-32' , 
    'openai_coca_ViT-L-14 laion2b_s13b_b90k':'coca_ViT-L-14'

}

In [8]:
dfSingle_deleted = dfSingle[dfSingle['model'].isin(dictModels.keys())]
avg_acc = dfSingle_deleted.groupby(['model'])['MRR' , 'Recall@1' , 'Recall@3'].mean().reset_index()
avg_acc

C:\Users\EA\AppData\Local\Temp\ipykernel_15188\2558110551.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  avg_acc = dfSingle_deleted.groupby(['model'])['MRR' , 'Recall@1' , 'Recall@3'].mean().reset_index()


,model,MRR,Recall@1,Recall@3
0,align_kakaobrain/align-base,0.384059,0.220169,0.419551
1,alt_BAAI/AltCLIP,0.464252,0.311553,0.511576
2,flava_facebook/flava-full,0.404092,0.216917,0.465387
3,openai_EVA01-g-14 laion400m_s11b_b41k,0.448361,0.267261,0.517774
4,openai_EVA02-L-14 merged2b_s4b_b131k,0.565203,0.391351,0.668413
5,openai_RN50x64 openai,0.423789,0.259109,0.467247
6,openai_ViT-B-16 openai,0.346936,0.187947,0.378354
7,openai_ViT-B-32 openai,0.256267,0.087661,0.255372
8,openai_ViT-L-14 openai,0.387477,0.215129,0.414825
9,openai_coca_ViT-B-32 laion2b_s13b_b90k,0.331850,0.179376,0.350313


In [9]:
dfTwo_deleted = dfTwo[dfTwo['model'].isin(dictModels.keys())]
avg_acc = dfTwo_deleted.groupby(['model'])['MRR' , 'Recall@1' , 'Recall@3'].mean().reset_index()
avg_acc

C:\Users\EA\AppData\Local\Temp\ipykernel_15188\2507834131.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  avg_acc = dfTwo_deleted.groupby(['model'])['MRR' , 'Recall@1' , 'Recall@3'].mean().reset_index()


,model,MRR,Recall@1,Recall@3
0,align_kakaobrain/align-base,0.371181,0.192195,0.413620
1,alt_BAAI/AltCLIP,0.407499,0.240235,0.450826
2,flava_facebook/flava-full,0.388336,0.185140,0.476246
3,openai_EVA01-g-14 laion400m_s11b_b41k,0.329555,0.144848,0.364745
4,openai_EVA02-L-14 merged2b_s4b_b131k,0.487924,0.300165,0.580082
5,openai_RN50x64 openai,0.326589,0.195874,0.302092
6,openai_ViT-B-16 openai,0.241856,0.090212,0.234892
7,openai_ViT-B-32 openai,0.203593,0.039899,0.178953
8,openai_ViT-L-14 openai,0.342756,0.195907,0.358861
9,openai_coca_ViT-B-32 laion2b_s13b_b90k,0.335718,0.155303,0.399680


In [10]:
dfTwo2_deleted = dfTwo2[dfTwo2['model'].isin(dictModels.keys())]
avg_acc = dfTwo2_deleted.groupby(['model'])['MRR' , 'Recall@1' , 'Recall@3'].mean().reset_index()
avg_acc

C:\Users\EA\AppData\Local\Temp\ipykernel_15188\1961913323.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  avg_acc = dfTwo2_deleted.groupby(['model'])['MRR' , 'Recall@1' , 'Recall@3'].mean().reset_index()


,model,MRR,Recall@1,Recall@3
0,align_kakaobrain/align-base,0.360911,0.177671,0.407309
1,alt_BAAI/AltCLIP,0.432642,0.268673,0.479279
2,flava_facebook/flava-full,0.388053,0.204491,0.442894
3,openai_EVA01-g-14 laion400m_s11b_b41k,0.348416,0.165758,0.395610
4,openai_EVA02-L-14 merged2b_s4b_b131k,0.493589,0.307482,0.585291
5,openai_RN50x64 openai,0.437257,0.277304,0.482637
6,openai_ViT-B-16 openai,0.335211,0.200058,0.347407
7,openai_ViT-B-32 openai,0.267689,0.103479,0.278876
8,openai_ViT-L-14 openai,0.370020,0.171953,0.453705
9,openai_coca_ViT-B-32 laion2b_s13b_b90k,0.342966,0.193204,0.354398
